In [3]:
# Visual decklists edition 2.0

import pandas as pd
import pdfkit
import os
import operator

# score calculator

class ScoreCalculator():
    def __init__(self, draft_name):
        self.draft_name = draft_name
        self.loadMatches()
        self.extractPlayers()
        self.calculateScores()
        self.prepareTableForCoverage()
        
    def loadMatches(self):
        filename = 'matches_'+self.draft_name+'.json'
        matches = pd.read_json('data/matches/'+filename)
        match_data = []
        for i in matches["matches"]: 
            record = []
            record.append({"player": i[0]["player"], "score": i[0]["score"]})
            record.append({"player": i[1]["player"], "score": i[1]["score"]})
            match_data.append(record)
        self.match_data = match_data    

    def extractPlayers(self):
        players = []
        for i in self.match_data:
            players.append(i[0]["player"])
            players.append(i[1]["player"])
        self.players = list(set(players))

    def normalizedScore(self, player_score):
        deals = player_score["won"] + player_score["lost"]
        win_rate =(player_score["won"]+0.0)/deals
        
        return player_score["points"]*100 + round(win_rate *100, 0)
    
    def getSortedList(self, player_points):
        standings = {}
        for i in self.players:
            score = self.normalizedScore(player_points[i])
            standings[i] = score
        sorted_standings = sorted(standings.items(), key=operator.itemgetter(1), reverse=True)

        return sorted_standings

    def calculateScores(self):
        player_points = {}
        for i in self.players:
            player_points[i] = {"points": 0, "won": 0, "lost": 0}

        for i in self.players:
            for j in self.match_data:
                if j[0]["player"] == i:
                    player_points[i]["won"] +=j[0]["score"]
                    if j[0]["score"] > j[1]["score"]:
                        player_points[i]["points"] += 3
                    player_points[i]["lost"] +=j[1]["score"]
                if j[1]["player"] == i:
                    player_points[i]["won"] +=j[1]["score"]
                    if j[1]["score"] > j[0]["score"]:
                        player_points[i]["points"] += 3
                    player_points[i]["lost"] +=j[0]["score"]

        self.standings = self.getSortedList(player_points)
        self.player_points = player_points
        
    def prepareTableForCoverage(self):
        table_string = ""

        my_scores = self.standings 
        my_points = self.player_points
        results = []
        for i in my_scores: 
            results.append(i[0])
        table_string += '|Pl.|Player|Points|W|L|\n'
        table_string += '|---|---|---|---|---|\n'

        for i in range(0, len(results)):
            table_string+= '|'+str(i+1)+'|'+results[i]+'|'+str(my_points[results[i]]['points'])+'|'
            table_string+= str(my_points[results[i]]['won'])
            table_string+= '|'+str(my_points[results[i]]['lost'])+'|\n'

        my_file = open('data/score_table.txt', 'w')
        my_file.write(table_string)
        my_file.close()
        
    def getStandings(self):
        return self.standings, self.player_points


class coverageModule():
    def __init__(self, draft_name):
        #self.all_cards = pd.read_json('data/cards/AllCards.json', orient='columns')
        self.template = 'data/deck_template.html'
        self.draft_name = draft_name
        self.sc = ScoreCalculator(self.draft_name)
        
    def cleanImageName(self, imagename):
        imagename = imagename.replace('`', '')
        imagename = imagename.replace(',', '')
        imagename = imagename.replace('\'', ' ')
        imagename = imagename.replace('-', ' ')
        imagename = imagename.replace('  ', ' ')
        imagename = imagename.lower()
        return imagename
    
    def cardsToImageNames(self, cards):
        my_images = []
        for c in cards:
            #my_images.append(self.all_cards[i]['imageName']+'.jpg')
            my_images.append(self.cleanImageName(c)+'.jpg')
        return my_images
    
    def createDeckContent(self, images):
        my_content = ''
        for i in images:
            my_content+='<img src=\"data/images_all/'+i+'\"/>'
        return my_content
    
    def createDeckHeader(self, name, player, draft):
        header = "<div><h1><b>[name]</b> built by <b>[player]</b> on draft <b>[draft]</b></h1></div>"
        header = header.replace('[name]', name).replace('[player]', player)
        draft = str(draft)
        draft = draft[0:2]+'_'+draft[3:]
        header = header.replace('[draft]', draft)
        return header
    
    def injectContentToTemplate(self, my_content, my_template, filename):
        template = open(my_template, 'r')
        t = template.read()
        t = t.replace('[content]', my_content)
        template.close()
        html_file = filename+'.html'
        my_html = open(html_file, 'w')
        my_html.write(t)
        my_html.close()
        self.convertHtmlToPdf(html_file, 'linux')
        
    def createBasicsSection(self, basics):
        basic_dict = {
            'Plains': 'plains.jpg',
            'Island': 'island.jpg',
            'Swamp': 'swamp.png',
            'Mountain': 'mountain.png',
            'Forest': 'forest.jpg',
            'plains': 'plains.jpg',
            'island': 'island.jpg',
            'swamp': 'swamp.png',
            'mountain': 'mountain.png',
            'forest': 'forest.jpg'
        }
        
        my_basics = "<table><tr>"
        for key in basics:
            if basics[key] > 0:
                my_basics+='<td><img src=\"data/images/'+basic_dict[key]+'\"/></td>'
        my_basics+='</tr><tr>'
        for key in basics:
            if basics[key] > 0:
                my_basics+='<td><h1><b>'+str(basics[key])+'</b></h1></td>'
        my_basics+='</tr></table>'
        return my_basics
    
    def compileOneDeck(self, header, content, basics):
        return '<div>'+header+content+basics+'</div>'
    
    def prepareScoreTable(self):
        standings, player_points = self.sc.getStandings()
        results = []
        
        for i in standings: 
            results.append(i[0])
            
        tab_string = '</p><h2>Final Standings:</h2><table style="width:600px">'
        tab_string+='<tr><th>Place </th><th>Player </th><th>Points </th><th>Won </th><th>Lost </th><th>Win %</th></tr>'
            
        acc = 1
        for r in results:
            line = '<tr><td>'+str(acc)+' </td><td>'+r+' </td><td>'+str(player_points[r]['points'])+' </td>'
            line+= '<td>'+str(player_points[r]['won'])+' </td><td>'+str(player_points[r]['lost'])+' </td>'
            win_rate = round((player_points[r]['won']+0.0)/(player_points[r]['won']+player_points[r]['lost']), 4)*100
            line+='<td>'+str(win_rate)+'%</td></tr>'
            
            tab_string+=line
            acc+=1
        
        tab_string+='</table>'    
        return tab_string
    
    def getRounds(self, l):
        if l == 6: return 3
        elif l==10: return 5
        elif l==15: return 5
        elif l==21: return 7
        else: return 8
        
    def prepareMatchesTable(self):
        matches = self.sc.match_data
        l = len(matches)
        rounds = self.getRounds(l)
        
        matches_string = '</p><h2>Matches:</h2><p><table style="width:600px"><th>Match</th><th>Round</th><th>Player #1</th>'
        matches_string+= '<th>Player #2</th><th>Score #1</th><th>Score #2</th>'
        acc = 1
        match_per_round = l/rounds
        r = 1
        racc=0
        for m in matches:
            one_match = '<tr><td>'+str(acc)+'</td><td>'+str(r)+'</td><td>'+m[0]['player']+'</td>'
            one_match+= '<td>'+m[1]['player']+'</td><td>'+str(m[0]['score'])+'</td><td>'+str(m[1]['score'])+'</td></tr>'
            
            racc+=1
            acc+=1
            if racc == match_per_round:
                r+=1
                racc=0
                
            matches_string+=one_match
        matches_string+='</table>'
        return matches_string
        
    def convertHtmlToPdf(self, html_file, operating_system):
        tmp = html_file.split('.')
        
        if operating_system == 'win':
            # windows only so far, maybe add version for linux later, ro a parameter
            path_wkthmltopdf = "C:\\Users\\Jacek\\Desktop\\MTG Scripts\\CoverageModule\\wkhtmltopdf\\bin\\wkhtmltopdf.exe"
            config = pdfkit.configuration(wkhtmltopdf = path_wkthmltopdf)
            pdfkit.from_file([html_file], 'data/coverage/'+tmp[0]+'.pdf', configuration=config)
        else:
            pdfkit.from_file(html_file, 'data/coverage/'+tmp[0]+'.pdf') 
        
    def visualiseAllDecks(self):
        filename = 'decklists_'+self.draft_name+'.json'
        df = pd.read_json('data/decks/'+filename, orient='columns')
        
        full_content = ""
        for i in range(0, len(df)):
            images = self.cardsToImageNames(sorted(df.cards[i]))
            content = self.createDeckContent(images)
            header = self.createDeckHeader(df.name[i], df.player[i], df.draft[i])
            basics = self.createBasicsSection(df.basic[i])
            compiled = self.compileOneDeck(header, content, basics)
            full_content+=compiled
            if i < len(df)-1:
                full_content+='<p style="page-break-after: always;">&nbsp;</p><p style="page-break-before: always;">&nbsp;</p>'
        
        full_content+= self.prepareMatchesTable()
        full_content+= self.prepareScoreTable()
        
        self.injectContentToTemplate(full_content, self.template, filename)
        
    def prepareDecklistsForCoverage(self):
        filename = 'decklists_'+self.draft_name+'.json'
        data = pd.read_json('data/decks/'+filename, orient='columns')
        
        decks_string = ''
        for i in range(0, len(data['player'])):
            decks_string+='Player:  '+data['player'][i]+'\n'
            decks_string+='Archetype: '+data['name'][i]+'\n'
            decks_string+='--------------------\n'
            decks_string+='Decklist ('+str(len(data['cards'][i]))+'):\n'
            for j in data['cards'][i]:
                decks_string+=j+'\n'
            decks_string+='--------------------\n'
            basics = data['basic'][i]
            basic_count = basics['Plains']+basics['Island']+basics['Swamp']+basics['Mountain']+basics['Forest']
            decks_string+='Basic lands ('+str(basic_count)+')\n'
            if basics['Plains'] > 0: decks_string+=str(basics['Plains'])+' Plains\n'
            if basics['Island'] > 0: decks_string+=str(basics['Island'])+' Island\n'
            if basics['Swamp'] > 0: decks_string+=str(basics['Swamp'])+' Swamp\n'
            if basics['Mountain'] > 0: decks_string+=str(basics['Mountain'])+' Mountain\n'
            if basics['Forest'] > 0: decks_string+=str(basics['Forest'])+' Forest\n'
            if i < len(data['player']) -1: decks_string +='\n*++++++++++++++++++++*\n'
                
        out_filename = 'coverage_'+filename.replace('json', 'txt')
        my_file = open('data/coverage/'+out_filename, 'w')
        my_file.write(decks_string)
        my_file.close()

cm = coverageModule('20_4')

#deckfiles = os.listdir('data/decks/')

#for d in deckfiles:
#    tmp = d.split('.')
#    if tmp[1] == 'json':
#        print(d)
#        coverage_module.visualiseAllDecks(tmp[0])

#cm.visualiseAllDecks()

cm.prepareDecklistsForCoverage()